# Loop Catalog hg38 to hg19

The focus of this notebook is to convert loop data from hg38 to hg19. This notebook starts from the raw Loop Catalog data and generates hg19 mcool files. Check out the `Loop-Catalog-Samples-and-Mcool-to-5kb-Cool` notebook for more details on converting to longrange files.

In [3]:
import os 
import numpy as np
import pandas as pd
import glob
import subprocess as sp
import json
import warnings
warnings.filterwarnings('ignore')

# change the working directory
os.chdir('/mnt/BioHome/jreyna/jreyna-temp/projects/dchallenge')

pd.set_option('display.max_columns', 1000)

outdir = 'results/main/loop_catalog/processing/hg38_to_hg19/liftover/'
os.makedirs(outdir, exist_ok=True)

In [4]:
# adding cooler to the path variable (won't work without)
os.environ['PATH'] += ':/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/software/mamba/envs/hichip-db/bin/'

In [5]:
pairLiftOver = "/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/software/mamba/envs/pairliftover/bin/pairLiftOver"
cooler_cli = '/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/software/mamba/envs/hichip-db/bin/cooler'

In [6]:
# getting the template paths for different loops
paths = {}
paths['HiCCUPS'] = '/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/hiccups/{sample_name}/postprocessed_pixels_{res}.bedpe'
paths['FitHiChIP using ChIP-Seq peaks'] = '/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/fithichip/{sample_name}_chipseq.peaks/{stg_str}/FitHiChIP_Peak2ALL_b{res}_L20000_U2000000/P2PBckgr_{stg_no}/Coverage_Bias/FitHiC_BiasCorr/FitHiChIP-{stg_str}.interactions_FitHiC_Q0.01.bed'
paths['FitHiChIP using HiChIP-Peaks peaks'] = '/mnt/BioAdHoc/Groups/vd-ay/kfetter/hichip-db-loop-calling/results/loops/fithichip/{sample_name}_hichip-peaks.peaks/{stg_str}/FitHiChIP_Peak2ALL_b{res}_L20000_U2000000/P2PBckgr_{stg_no}/Coverage_Bias/FitHiC_BiasCorr/FitHiChIP-{stg_str}.interactions_FitHiC_Q0.01.bed'

In [7]:
# loading sample data
samples = pd.read_excel('results/main/loop_catalog/master-table.immune-samples-only.xlsx')
cline, gse, species, pulldown, biorep = zip(*samples['full.sample.name'].str.split('.'))

samples['cline'] = cline
samples['gse'] = gse
samples['pulldown'] = pulldown
samples['biorep'] = biorep

In [8]:
samples.pulldown.value_counts()

H3K27ac    22
CTCF       11
SMC1A       2
H3K4me1     1
KLF4        1
NANOG       1
OCT4        1
Rad21       1
Name: pulldown, dtype: int64

In [9]:
enhancer_samples = samples.loc[samples['pulldown'] == 'H3K27ac']

In [10]:
enhancer_samples.cline.unique().tolist()

['GM12878',
 'HARA',
 'MyLa',
 'Naive_Tcells',
 'Th17',
 'Treg',
 'lgs000379',
 'lgs101645',
 'lgs102580',
 'lgs102943',
 'lgs301283',
 'lgs301315']

## Lifting over HiCCUPs files

In [14]:
def shorten_and_int_format(x):
    s = '{:.2f}'.format(x)
    s = int(float(s) * 100)
    return(s)

In [13]:
# extra FitHiChIP loops for best filtering parameters
res = 5000
stg_no = 1
stg_str = 'S5'

chromsizes_hg38 = 'results/refs/hg38/hg38.chrom.sizes'
chromsizes_hg19 = 'results/refs/hg19/hg19.chrom.sizes'
chain_file = 'results/refs/ucsc/hg38ToHg19.over.chain.gz'   

In [9]:
hiccups_dir = os.path.join(outdir, 'hiccups/')
os.makedirs(hiccups_dir, exist_ok=True)

In [12]:
for sample_name in enhancer_samples['full.sample.name']:
    
    print(sample_name)
    
    # get current possible loop file
    hiccups = paths['HiCCUPS'].format(sample_name=sample_name,
                                      stg_str=stg_str,
                                      stg_no=stg_no,
                                      res=res)
    
    # output data
    liftover_hg19_cool = os.path.join(hiccups_dir, '{}.hg19.cool'.format(sample_name))

    # run sample if the input are present and output are not
    if os.path.exists(hiccups) and not os.path.exists(liftover_hg19_cool):
        
        hiccups_df = pd.read_table(hiccups)
        hiccups_df = hiccups_df.loc[~hiccups_df['#chr1'].str.match('^#')]
        hiccups_df.loc[:, '#chr1'] = 'chr' + hiccups_df.loc[:, '#chr1']
        hiccups_df.loc[:, 'chr2'] = 'chr' + hiccups_df.loc[:, 'chr2']
        hiccups_df.loc[:, 'x1'] = hiccups_df['x1'].astype(int)
        hiccups_df.loc[:, 'x2'] = hiccups_df['x2'].astype(int)
        hiccups_df.loc[:, 'y1'] = hiccups_df['y1'].astype(int)
        hiccups_df.loc[:, 'y2'] = hiccups_df['y2'].astype(int)

        hiccups_lr = hiccups_df[['#chr1', 'x1', 'x2', 'chr2', 'y1', 'y2', 'fdrH']]
        
        #hiccups_lr.loc[hiccups_lr['fdrH'] == 0, 'fdrH'] = 1 * 10**(-8) # assign super sign to 0 p-value regions
        hiccups_lr['-log10(fdrH)'] =  -1 * np.log10(hiccups_lr['fdrH'])

        # assign max score to inf
        s = hiccups_lr['-log10(fdrH)']
        s = s[s != np.inf]     
        hiccups_lr.loc[hiccups_lr['-log10(fdrH)'] == np.inf, '-log10(fdrH)'] = max(s)
        hiccups_lr['-log10(fdrH)_100tmp'] = hiccups_lr['-log10(fdrH)'].apply(shorten_and_int_format)

        # save for conversion
        bg2_df = hiccups_lr[['#chr1', 'x1', 'x2', 'chr2', 'y1', 'y2', '-log10(fdrH)_100tmp']]
        bg2_fn = os.path.join(hiccups_dir, '{}.bed'.format(sample_name))
        bg2_df.to_csv(bg2_fn, sep='\t', header=False, index=False)
        
        
        # convert the pairs format into cool
        cool_fn = os.path.join(hiccups_dir, '{}.hg38.cool'.format(sample_name))
        #make_cooler_cmd = '{} load --count-as-float --format bg2 {}:5000 {} {}'.format(cooler_cli,chromsizes_hg38,bg2_fn, cool_fn)
        make_cooler_cmd = '{} load --format bg2 {}:5000 {} {}'.format(cooler_cli,chromsizes_hg38,bg2_fn, cool_fn)
        sp.check_output(make_cooler_cmd, shell=True)

        
        # liftover the cool program   
        liftover_hg19_prefix = os.path.join(hiccups_dir, '{}.hg19'.format(sample_name))
        plo_cmd = """{} --input {} \
                        --input-format cooler \
                        --out-pre {} \
                        --output-format cool \
                        --in-assembly hg38 \
                        --out-assembly hg19 \
                        --chain-file {} \
                        --out-chromsizes {} \
                        --tmpdir {}"""
        plo_cmd = plo_cmd.format(pairLiftOver, cool_fn, liftover_hg19_prefix, chain_file, chromsizes_hg19, outdir)
        sp.check_output(plo_cmd, shell=True)

GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.cli.load:fields: {'chrom1': 0, 'start1': 1, 'end1': 2, 'chrom2': 3, 'start2': 4, 'end2': 5, 'count': 6}
INFO:cooler.cli.load:dtypes: {'chrom1': <class 'str'>, 'start1': <class 'int'>, 'end1': <class 'int'>, 'chrom2': <class 'str'>, 'start2': <class 'int'>, 'end2': <class 'int'>, 'count': <class 'numpy.int32'>}
INFO:cooler.cli.load:symmetric-upper: True
INFO:cooler.create:Writing chunk 0: /mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/hiccups/tmpw3q5cu06.multi.cool::0
INFO:cooler.create:Creating cooler at "/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/hiccups/tmpw3q5cu06.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Merging int

INFO:cooler.create:Writing pixels
INFO:cooler.create:Binning chr1:0-20775000|*
INFO:cooler.create:Binning chr1:20775000-41550000|*
INFO:cooler.create:Binning chr1:41550000-62325000|*
INFO:cooler.create:Binning chr1:62325000-83100000|*
INFO:cooler.create:Binning chr1:83100000-103875000|*
INFO:cooler.create:Binning chr1:103875000-124650000|*
INFO:cooler.create:Binning chr1:124650000-145425000|*
INFO:cooler.create:Binning chr1:145425000-166200000|*
INFO:cooler.create:Finished chr1:0-20775000|*
INFO:cooler.create:Binning chr1:166200000-186975000|*
INFO:cooler.create:Finished chr1:124650000-145425000|*
INFO:cooler.create:Binning chr1:186975000-207750000|*
INFO:cooler.create:Finished chr1:83100000-103875000|*
INFO:cooler.create:Binning chr1:207750000-228525000|*
INFO:cooler.create:Finished chr1:103875000-124650000|*
INFO:cooler.create:Binning chr1:228525000-249250621|*
INFO:cooler.create:Finished chr1:41550000-62325000|*
INFO:cooler.create:Binning chr2:0-21290000|*
INFO:cooler.create:Finishe

INFO:cooler.create:Finished chr11:0-38350000|*
INFO:cooler.create:Binning chr13:89910000-115169878|*
INFO:cooler.create:Finished chr11:38350000-76700000|*
INFO:cooler.create:Binning chr14:0-48230000|*
INFO:cooler.create:Finished chr12:116040000-133851895|*
INFO:cooler.create:Binning chr14:48230000-96460000|*
INFO:cooler.create:Finished chr12:38680000-77360000|*
INFO:cooler.create:Binning chr14:96460000-107349540|*
INFO:cooler.create:Finished chr13:44955000-89910000|*
INFO:cooler.create:Binning chr15:0-50495000|*
INFO:cooler.create:Finished chr12:0-38680000|*
INFO:cooler.create:Binning chr15:50495000-100990000|*
INFO:cooler.create:Finished chr14:96460000-107349540|*
INFO:cooler.create:Binning chr15:100990000-102531392|*
INFO:cooler.create:Finished chr15:100990000-102531392|*
INFO:cooler.create:Binning chr16:0-57300000|*
INFO:cooler.create:Finished chr13:89910000-115169878|*
INFO:cooler.create:Binning chr16:57300000-90354753|*
INFO:cooler.create:Finished chr12:77360000-116040000|*
INFO:c

  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

INFO:cooler.balance:variance is 1034389.0434782609
INFO:cooler.balance:variance is 1563503328.4061358
INFO:cooler.balance:variance is 14593660492.824753
INFO:cooler.balance:variance is 25101654036108.664
INFO:cooler.balance:variance is 233927806446816.34
INFO:cooler.balance:variance is 4.0346896541842406e+17
INFO:cooler.balance:variance is 3.75968764539847e+18
INFO:cooler.balance:variance is 6.485540715193762e+21
INFO:cooler.balance:variance is 6.043458118510244e+22
INFO:cooler.balance:variance is 1.0425189804032787e+26
INFO:cooler.balance:variance is 9.714560430380961e+26
INFO:cooler.balance:variance is 1.6757987353138976e+30
INFO:cooler.balance:variance is 1.5615684754566194e+31
INFO:cooler.balance:variance is 2.6937652909451416e+34
INFO:cooler.balance:variance is 2.510145678339186e+35
INFO:cooler.balance:variance is 4.330097222761319e+38
INFO:cooler.balance:variance is 4.034937589464713e+39
INF

...

INFO:cooler.create:Finished chr11:115050000-135006516|*
INFO:cooler.create:Binning chr12:77360000-116040000|*
INFO:cooler.create:Finished chr10:76400000-114600000|*
INFO:cooler.create:Binning chr12:116040000-133851895|*
INFO:cooler.create:Finished chr11:38350000-76700000|*
INFO:cooler.create:Binning chr13:0-44955000|*
INFO:cooler.create:Finished chr11:0-38350000|*
INFO:cooler.create:Binning chr13:44955000-89910000|*
INFO:cooler.create:Finished chr10:38200000-76400000|*
INFO:cooler.create:Binning chr13:89910000-115169878|*
INFO:cooler.create:Finished chr11:76700000-115050000|*
INFO:cooler.create:Binning chr15:0-50495000|*
INFO:cooler.create:Finished chr12:116040000-133851895|*
INFO:cooler.create:Binning chr15:50495000-100990000|*
INFO:cooler.create:Finished chr12:0-38680000|*
INFO:cooler.create:Binning chr15:100990000-102531392|*
INFO:cooler.create:Finished chr15:100990000-102531392|*
INFO:cooler.create:Binning chr16:0-57300000|*
INFO:cooler.create:Finished chr13:44955000-89910000|*
INF

INFO:cooler.cli.zoomify:Balancing zoom level with bin size 5000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/hiccups/Treg.GSE101498.Homo_Sapiens.H3K27ac.b3.hg19.mcool::resolutions/5000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

INFO:cooler.balance:variance is 117619.88888888889
INFO:cooler.balance:variance is 2038612.5379369883
INFO:cooler.balance:variance is 37056390.302831106
INFO:cooler.balance:variance is 673583644.131497
INFO:cooler.balance:variance is 12243905084.484262
INFO:cooler.balance:variance is 222560647105.90482
INFO:cooler.balance:variance is 4045542765842.6313
INFO:cooler.balance:variance is 73536883016312.2
INFO:cooler.balance:variance is 1336699048002386.2
INFO:cooler.balance:variance is 2.429752624317976e+16
INFO:cooler.balance:variance is 4.4166245380385363e+17
INFO:cooler.balance:variance is 8.028213290017353e+18
INFO:cooler.balance:variance is 1.4593092094406348e+20
IN

INFO:cooler.balance:variance is 1.3312413708746678e+180
INFO:cooler.balance:variance is 2.4198320626600782e+181
INFO:cooler.balance:variance is 4.3985916750997785e+182
INFO:cooler.balance:variance is 7.995434486056266e+183
INFO:cooler.balance:variance is 1.4533509209937696e+185
INFO:cooler.balance:variance is 2.641793767727176e+186
INFO:cooler.balance:variance is 4.802057239162859e+187
INFO:cooler.balance:variance is 8.728824335154491e+188
INFO:cooler.balance:variance is 1.5866611012589225e+190
INFO:cooler.balance:variance is 2.884115149515857e+191
INFO:cooler.balance:variance is 5.242531117115647e+192
INFO:cooler.balance:variance is 9.529485158919355e+193
INFO:cooler.balance:variance is 1.7321993015470537e+195
INFO:cooler.balance:variance is 3.1486637213256954e+196
INFO:cooler.balance:variance is 5.723407936452903e+197
INFO:cooler.balance:variance is 1.0403587460035286e+199
INFO:cooler.balance:variance is 1.8910871501793076e+200
INFO:cooler.balance:variance is 3.437478296127254e+201
I

  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

  scale = nzmarg.mean()

INFO:cooler.cli.zoomify:Balancing zoom level with bin size 1000000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/hiccups/Treg.GSE101498.Homo_Sapiens.H3K27ac.b3.hg19.mcool::resolutions/1000000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

  scale = nzmarg.mean()

INFO:cooler.cli.zoomify:Balancing zoom level with bin size 2500000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/hiccups/Treg.GSE101498.Homo_Sapiens.H3K27ac.b3.hg19.mcool::resolutions/2500000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

  scale = nzmarg.mean()

pairLiftOver.utilities    INFO    @ 09/22/22 10:33:50: Done


lgs000379.GSE116193.Homo_Sapiens.H3K27ac.b1
lgs101645.GSE116193.Homo_Sapiens.H3K27ac.b1
lgs102580.GSE116193.Homo_Sapiens.H3K27ac.b1
lgs102943.GSE116193.Homo_Sapiens.H3K27ac.b1
lgs301283.GSE116193.Homo_Sapiens.H3K27ac.b1
lgs301315.GSE116193.Homo_Sapiens.H3K27ac.b1


## Lifting over FitHiChIP using ChIP-Seq peaks loops

In [14]:
fit_cp_dir = os.path.join(outdir, 'fit_cp/')
os.makedirs(fit_cp_dir, exist_ok=True)

In [15]:
# processing data from FitHiChIP using ChIP-Seq peaks
for sample_name in enhancer_samples['full.sample.name']:

    print(sample_name)
    
    # input data
    fit_cp = paths['FitHiChIP using ChIP-Seq peaks'].format(sample_name=sample_name,
                                      stg_str=stg_str,
                                      stg_no=stg_no,
                                      res=res)
    
    # output data
    liftover_hg19_cool = os.path.join(fit_cp_dir, '{}.hg19.cool'.format(sample_name))

    # run sample if the input are present and output are not
    if os.path.exists(fit_cp) and not os.path.exists(liftover_hg19_cool):
        
        fit_cp_df = pd.read_table(fit_cp)
                
        fit_cp_lr = fit_cp_df[['chr1', 's1', 'e1', 'chr2', 's2', 'e2', 'p']]
        
        fit_cp_lr['-log10(p)'] =  -1 * np.log10(fit_cp_lr['p'])

        # assign max score to inf
        s = fit_cp_lr['-log10(p)']
        s = s[s != np.inf]     
        fit_cp_lr.loc[fit_cp_lr['-log10(p)'] == np.inf, '-log10(p)'] = max(s)
        
        fit_cp_lr['-log10(p)_100tmp'] = fit_cp_lr['-log10(p)'].apply(shorten_and_int_format)

        # save for conversion
        bg2_df = fit_cp_lr[['chr1', 's1', 'e1', 'chr2', 's2', 'e2', '-log10(p)_100tmp']]
        bg2_fn = os.path.join(fit_cp_dir, '{}.bed'.format(sample_name))
        bg2_df.to_csv(bg2_fn, sep='\t', header=False, index=False)
        
        # convert the pairs format into cool
        cool_fn = os.path.join(fit_cp_dir, '{}.hg38.cool'.format(sample_name))
        #make_cooler_cmd = '{} load --count-as-float --format bg2 {}:5000 {} {}'.format(cooler_cli,chromsizes_hg38,bg2_fn, cool_fn)
        make_cooler_cmd = '{} load --format bg2 {}:5000 {} {}'.format(cooler_cli,chromsizes_hg38,bg2_fn, cool_fn)
        sp.check_output(make_cooler_cmd, shell=True)
        
        # liftover the cool program   
        liftover_hg19_prefix = os.path.join(fit_cp_dir, '{}.hg19'.format(sample_name))
        plo_cmd = """{} --input {} \
                        --input-format cooler \
                        --out-pre {} \
                        --output-format cool \
                        --in-assembly hg38 \
                        --out-assembly hg19 \
                        --chain-file {} \
                        --out-chromsizes {} \
                        --tmpdir {}"""
        
        try: 
            plo_cmd = plo_cmd.format(pairLiftOver, cool_fn, liftover_hg19_prefix, chain_file, chromsizes_hg19, outdir)
            sp.check_output(plo_cmd, shell=True)
            
        except CalledProcessError as e:
            print('Problem with {}'.format(e))


GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1
GM12878.GSE101498.Homo_Sapiens.H3K27ac.b2
HARA.GSE147854.Homo_Sapiens.H3K27ac.b1
MyLa.GSE101498.Homo_Sapiens.H3K27ac.b1
MyLa.GSE101498.Homo_Sapiens.H3K27ac.b2
MyLa.GSE137906.Homo_Sapiens.H3K27ac.b1
MyLa.GSE151193.Homo_Sapiens.H3K27ac.b1
Naive_Tcells.GSE101498.Homo_Sapiens.H3K27ac.b1
Naive_Tcells.GSE101498.Homo_Sapiens.H3K27ac.b2
Naive_Tcells.GSE101498.Homo_Sapiens.H3K27ac.b3
Th17.GSE101498.Homo_Sapiens.H3K27ac.b1
Th17.GSE101498.Homo_Sapiens.H3K27ac.b2
Th17.GSE101498.Homo_Sapiens.H3K27ac.b3
Treg.GSE101498.Homo_Sapiens.H3K27ac.b1
Treg.GSE101498.Homo_Sapiens.H3K27ac.b2
Treg.GSE101498.Homo_Sapiens.H3K27ac.b3
lgs000379.GSE116193.Homo_Sapiens.H3K27ac.b1


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.cli.load:fields: {'chrom1': 0, 'start1': 1, 'end1': 2, 'chrom2': 3, 'start2': 4, 'end2': 5, 'count': 6}
INFO:cooler.cli.load:dtypes: {'chrom1': <class 'str'>, 'start1': <class 'int'>, 'end1': <class 'int'>, 'chrom2': <class 'str'>, 'start2': <class 'int'>, 'end2': <class 'int'>, 'count': <class 'numpy.int32'>}
INFO:cooler.cli.load:symmetric-upper: True
INFO:cooler.create:Writing chunk 0: /mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/tmpqcx_4v7c.multi.cool::0
INFO:cooler.create:Creating cooler at "/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/tmpqcx_4v7c.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Merging into 

INFO:cooler.create:Creating cooler at "/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/lgs000379.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.5kb.cool::/"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Binning chr1:0-20775000|*
INFO:cooler.create:Binning chr1:20775000-41550000|*
INFO:cooler.create:Binning chr1:41550000-62325000|*
INFO:cooler.create:Binning chr1:62325000-83100000|*
INFO:cooler.create:Binning chr1:83100000-103875000|*
INFO:cooler.create:Binning chr1:103875000-124650000|*
INFO:cooler.create:Binning chr1:124650000-145425000|*
INFO:cooler.create:Binning chr1:145425000-166200000|*
INFO:cooler.create:Finished chr1:83100000-103875000|*
INFO:cooler.create:Binning chr1:166200000-186975000|*
INFO:cooler.create:Finished chr1:62325000-83100000|*
INFO:cooler.create:Binning chr1:186975000-207750000|*
INFO:cooler.create:Finished chr1:124650000-1454250

INFO:cooler.create:Finished chr9:73330000-109995000|*
INFO:cooler.create:Binning chr12:0-38680000|*
INFO:cooler.create:Finished chr9:109995000-141213431|*
INFO:cooler.create:Binning chr12:38680000-77360000|*
INFO:cooler.create:Finished chr10:114600000-135534747|*
INFO:cooler.create:Binning chr12:77360000-116040000|*
INFO:cooler.create:Finished chr11:115050000-135006516|*
INFO:cooler.create:Binning chr12:116040000-133851895|*
INFO:cooler.create:Finished chr10:76400000-114600000|*
INFO:cooler.create:Binning chr13:0-44955000|*
INFO:cooler.create:Finished chr11:38350000-76700000|*
INFO:cooler.create:Binning chr13:44955000-89910000|*
INFO:cooler.create:Finished chr11:0-38350000|*
INFO:cooler.create:Binning chr13:89910000-115169878|*
INFO:cooler.create:Finished chr12:116040000-133851895|*
INFO:cooler.create:Binning chr14:0-48230000|*
INFO:cooler.create:Finished chr11:76700000-115050000|*
INFO:cooler.create:Binning chr14:48230000-96460000|*
INFO:cooler.create:Finished chr13:89910000-115169878

  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.create:Creating cooler at "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/lgs000379.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.mcool::/resolutions/2500000"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.reduce:0 190
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.cli.zoomify:Balancing zoom level with bin size 5000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/lgs000379.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.mcool::resolutions/5000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

  scale = nzmarg.mean()

INFO:cooler.cli.zoomify:Balancing zoom level with bin size 10000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/lgs000379.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.m

...

lgs301315.GSE116193.Homo_Sapiens.H3K27ac.b1


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.cli.load:fields: {'chrom1': 0, 'start1': 1, 'end1': 2, 'chrom2': 3, 'start2': 4, 'end2': 5, 'count': 6}
INFO:cooler.cli.load:dtypes: {'chrom1': <class 'str'>, 'start1': <class 'int'>, 'end1': <class 'int'>, 'chrom2': <class 'str'>, 'start2': <class 'int'>, 'end2': <class 'int'>, 'count': <class 'numpy.int32'>}
INFO:cooler.cli.load:symmetric-upper: True
INFO:cooler.create:Writing chunk 0: /mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/tmpyssz4bax.multi.cool::0
INFO:cooler.create:Creating cooler at "/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/tmpyssz4bax.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Merging into 

INFO:cooler.create:Writing pixels
INFO:cooler.create:Binning chr1:0-20775000|*
INFO:cooler.create:Binning chr1:20775000-41550000|*
INFO:cooler.create:Binning chr1:41550000-62325000|*
INFO:cooler.create:Binning chr1:62325000-83100000|*
INFO:cooler.create:Binning chr1:83100000-103875000|*
INFO:cooler.create:Binning chr1:103875000-124650000|*
INFO:cooler.create:Binning chr1:124650000-145425000|*
INFO:cooler.create:Binning chr1:145425000-166200000|*
INFO:cooler.create:Finished chr1:62325000-83100000|*
INFO:cooler.create:Binning chr1:166200000-186975000|*
INFO:cooler.create:Finished chr1:83100000-103875000|*
INFO:cooler.create:Binning chr1:186975000-207750000|*
INFO:cooler.create:Finished chr1:41550000-62325000|*
INFO:cooler.create:Binning chr1:207750000-228525000|*
INFO:cooler.create:Finished chr1:103875000-124650000|*
INFO:cooler.create:Binning chr1:228525000-249250621|*
INFO:cooler.create:Finished chr1:0-20775000|*
INFO:cooler.create:Binning chr2:0-21290000|*
INFO:cooler.create:Finished 

INFO:cooler.create:Finished chr10:76400000-114600000|*
INFO:cooler.create:Binning chr12:77360000-116040000|*
INFO:cooler.create:Finished chr11:115050000-135006516|*
INFO:cooler.create:Binning chr12:116040000-133851895|*
INFO:cooler.create:Finished chr10:38200000-76400000|*
INFO:cooler.create:Binning chr14:0-48230000|*
INFO:cooler.create:Finished chr11:38350000-76700000|*
INFO:cooler.create:Binning chr14:48230000-96460000|*
INFO:cooler.create:Finished chr12:116040000-133851895|*
INFO:cooler.create:Binning chr14:96460000-107349540|*
INFO:cooler.create:Finished chr11:76700000-115050000|*
INFO:cooler.create:Binning chr15:0-50495000|*
INFO:cooler.create:Finished chr11:0-38350000|*
INFO:cooler.create:Binning chr15:50495000-100990000|*
INFO:cooler.create:Finished chr14:96460000-107349540|*
INFO:cooler.create:Binning chr15:100990000-102531392|*
INFO:cooler.create:Finished chr12:38680000-77360000|*
INFO:cooler.create:Binning chr16:0-57300000|*
INFO:cooler.create:Finished chr15:100990000-1025313

  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

INFO:cooler.balance:variance is 7399.25
INFO:cooler.balance:variance is 38.27386301829634
INFO:cooler.balance:variance is 0.19782445158330206
INFO:cooler.balance:variance is 0.0010224825410132996
INFO:cooler.balance:variance is 5.284839748593067e-06
INFO:cooler.cli.zoomify:Balancing zoom level with bin size 10000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/lgs301315.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.mcool::resolutions/10000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

  scale = nzmarg.mean()

INFO:cooler.cli.zoomify:Balancing zoom level with bin size 25000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_cp/lgs301315.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.mcool::resolutions/25000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret

## Lifting over FitHiChIP using ChIP-Seq peaks loops

In [16]:
fit_hp_dir = os.path.join(outdir, 'fit_hp/')
os.makedirs(fit_hp_dir, exist_ok=True)

In [17]:
# processing data from FitHiChIP using ChIP-Seq peaks
for sample_name in enhancer_samples['full.sample.name']:

    print(sample_name)
    
    # input data
    fit_hp = paths['FitHiChIP using HiChIP-Peaks peaks'].format(sample_name=sample_name,
                                      stg_str=stg_str,
                                      stg_no=stg_no,
                                      res=res)
    
    # output data
    liftover_hg19_cool = os.path.join(fit_hp_dir, '{}.hg19.cool'.format(sample_name))

    # run sample if the input are present and output are not
    if os.path.exists(fit_hp) and not os.path.exists(liftover_hg19_cool):
        
        fit_hp_df = pd.read_table(fit_hp)
                
        fit_hp_lr = fit_hp_df[['chr1', 's1', 'e1', 'chr2', 's2', 'e2', 'p']]
        
        fit_hp_lr['-log10(p)'] =  -1 * np.log10(fit_hp_lr['p'])

        # assign max score to inf
        s = fit_hp_lr['-log10(p)']
        s = s[s != np.inf]     
        fit_hp_lr.loc[fit_hp_lr['-log10(p)'] == np.inf, '-log10(p)'] = max(s)
        
        fit_hp_lr['-log10(p)_100tmp'] = fit_hp_lr['-log10(p)'].apply(shorten_and_int_format)

        # save for conversion
        bg2_df = fit_hp_lr[['chr1', 's1', 'e1', 'chr2', 's2', 'e2', '-log10(p)_100tmp']]
        bg2_fn = os.path.join(fit_hp_dir, '{}.bed'.format(sample_name))
        bg2_df.to_csv(bg2_fn, sep='\t', header=False, index=False)
        
        # convert the pairs format into cool
        cool_fn = os.path.join(fit_hp_dir, '{}.hg38.cool'.format(sample_name))
        #make_cooler_cmd = '{} load --count-as-float --format bg2 {}:5000 {} {}'.format(cooler_cli,chromsizes_hg38,bg2_fn, cool_fn)
        make_cooler_cmd = '{} load --format bg2 {}:5000 {} {}'.format(cooler_cli,chromsizes_hg38,bg2_fn, cool_fn)
        sp.check_output(make_cooler_cmd, shell=True)
        
        # liftover the cool program   
        liftover_hg19_prefix = os.path.join(fit_hp_dir, '{}.hg19'.format(sample_name))
        plo_cmd = """{} --input {} \
                        --input-format cooler \
                        --out-pre {} \
                        --output-format cool \
                        --in-assembly hg38 \
                        --out-assembly hg19 \
                        --chain-file {} \
                        --out-chromsizes {} \
                        --tmpdir {}"""
        
        try: 
            plo_cmd = plo_cmd.format(pairLiftOver, cool_fn, liftover_hg19_prefix, chain_file, chromsizes_hg19, outdir)
            sp.check_output(plo_cmd, shell=True)
            
        except CalledProcessError as e:
            print('Problem with {}'.format(e))


GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.cli.load:fields: {'chrom1': 0, 'start1': 1, 'end1': 2, 'chrom2': 3, 'start2': 4, 'end2': 5, 'count': 6}
INFO:cooler.cli.load:dtypes: {'chrom1': <class 'str'>, 'start1': <class 'int'>, 'end1': <class 'int'>, 'chrom2': <class 'str'>, 'start2': <class 'int'>, 'end2': <class 'int'>, 'count': <class 'numpy.int32'>}
INFO:cooler.cli.load:symmetric-upper: True
INFO:cooler.create:Writing chunk 0: /mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/tmph48v4bjf.multi.cool::0
INFO:cooler.create:Creating cooler at "/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/tmph48v4bjf.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Merging into 

INFO:cooler.create:Writing pixels
INFO:cooler.create:Binning chr1:0-20775000|*
INFO:cooler.create:Binning chr1:20775000-41550000|*
INFO:cooler.create:Binning chr1:41550000-62325000|*
INFO:cooler.create:Binning chr1:62325000-83100000|*
INFO:cooler.create:Binning chr1:83100000-103875000|*
INFO:cooler.create:Binning chr1:103875000-124650000|*
INFO:cooler.create:Binning chr1:124650000-145425000|*
INFO:cooler.create:Binning chr1:145425000-166200000|*
INFO:cooler.create:Finished chr1:124650000-145425000|*
INFO:cooler.create:Binning chr1:166200000-186975000|*
INFO:cooler.create:Finished chr1:20775000-41550000|*
INFO:cooler.create:Binning chr1:186975000-207750000|*
INFO:cooler.create:Finished chr1:62325000-83100000|*
INFO:cooler.create:Binning chr1:207750000-228525000|*
INFO:cooler.create:Finished chr1:83100000-103875000|*
INFO:cooler.create:Finished chr1:41550000-62325000|*
INFO:cooler.create:Finished chr1:103875000-124650000|*
INFO:cooler.create:Binning chr1:228525000-249250621|*
INFO:cooler

INFO:cooler.create:Binning chr12:38680000-77360000|*
INFO:cooler.create:Binning chr12:77360000-116040000|*
INFO:cooler.create:Finished chr10:38200000-76400000|*
INFO:cooler.create:Binning chr12:116040000-133851895|*
INFO:cooler.create:Finished chr11:38350000-76700000|*
INFO:cooler.create:Binning chr13:0-44955000|*
INFO:cooler.create:Finished chr11:115050000-135006516|*
INFO:cooler.create:Binning chr13:44955000-89910000|*
INFO:cooler.create:Finished chr12:0-38680000|*
INFO:cooler.create:Binning chr13:89910000-115169878|*
INFO:cooler.create:Finished chr12:116040000-133851895|*
INFO:cooler.create:Binning chr14:0-48230000|*
INFO:cooler.create:Finished chr11:0-38350000|*
INFO:cooler.create:Binning chr14:48230000-96460000|*
INFO:cooler.create:Finished chr11:76700000-115050000|*
INFO:cooler.create:Binning chr14:96460000-107349540|*
INFO:cooler.create:Finished chr14:96460000-107349540|*
INFO:cooler.create:Binning chr15:0-50495000|*
INFO:cooler.create:Finished chr12:77360000-116040000|*
INFO:co

  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.create:Creating cooler at "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1.hg19.mcool::/resolutions/2500000"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.reduce:0 5686
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.cli.zoomify:Balancing zoom level with bin size 5000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/GM12878.GSE101498.Homo_Sapiens.H3K27ac.b1.hg19.mcool::resolutions/5000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

INFO:cooler.balance:variance is 5930934.434483529
INFO:cooler.balance:variance is 30209687.013491377
INFO:cooler.balance:variance is 24682506.252763983
INFO:cooler.balance:variance is 259309573.4355894
INFO:cooler.balance:variance is 207173725.

...

lgs301315.GSE116193.Homo_Sapiens.H3K27ac.b1


  is_cat = pd.api.types.is_categorical(bins["chrom"])

INFO:cooler.cli.load:fields: {'chrom1': 0, 'start1': 1, 'end1': 2, 'chrom2': 3, 'start2': 4, 'end2': 5, 'count': 6}
INFO:cooler.cli.load:dtypes: {'chrom1': <class 'str'>, 'start1': <class 'int'>, 'end1': <class 'int'>, 'chrom2': <class 'str'>, 'start2': <class 'int'>, 'end2': <class 'int'>, 'count': <class 'numpy.int32'>}
INFO:cooler.cli.load:symmetric-upper: True
INFO:cooler.create:Writing chunk 0: /mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/tmpu3wepfj5.multi.cool::0
INFO:cooler.create:Creating cooler at "/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/tmpu3wepfj5.multi.cool::/0"
INFO:cooler.create:Writing chroms
INFO:cooler.create:Writing bins
INFO:cooler.create:Writing pixels
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.create:Merging into 

INFO:cooler.create:Writing pixels
INFO:cooler.create:Binning chr1:0-20775000|*
INFO:cooler.create:Binning chr1:20775000-41550000|*
INFO:cooler.create:Binning chr1:41550000-62325000|*
INFO:cooler.create:Binning chr1:62325000-83100000|*
INFO:cooler.create:Binning chr1:83100000-103875000|*
INFO:cooler.create:Binning chr1:103875000-124650000|*
INFO:cooler.create:Binning chr1:124650000-145425000|*
INFO:cooler.create:Binning chr1:145425000-166200000|*
INFO:cooler.create:Finished chr1:0-20775000|*
INFO:cooler.create:Binning chr1:166200000-186975000|*
INFO:cooler.create:Finished chr1:41550000-62325000|*
INFO:cooler.create:Binning chr1:186975000-207750000|*
INFO:cooler.create:Finished chr1:166200000-186975000|*
INFO:cooler.create:Binning chr1:207750000-228525000|*
INFO:cooler.create:Finished chr1:20775000-41550000|*
INFO:cooler.create:Binning chr1:228525000-249250621|*
INFO:cooler.create:Finished chr1:103875000-124650000|*
INFO:cooler.create:Binning chr2:0-21290000|*
INFO:cooler.create:Finished

INFO:cooler.create:Finished chr12:77360000-116040000|*
INFO:cooler.create:Binning chr19:0-59128983|*
INFO:cooler.create:Finished chr18:66310000-78077248|*
INFO:cooler.create:Binning chr22:0-51304566|*
INFO:cooler.create:Finished chr14:0-48230000|*
INFO:cooler.create:Binning chr21:0-48129895|*
INFO:cooler.create:Finished chr14:48230000-96460000|*
INFO:cooler.create:Finished chr16:0-57300000|*
INFO:cooler.create:Finished chr20:0-63025520|*
INFO:cooler.create:Finished chr17:0-63765000|*
INFO:cooler.create:Finished chr21:0-48129895|*
INFO:cooler.create:Finished chr18:0-66310000|*
INFO:cooler.create:Finished chr22:0-51304566|*
INFO:cooler.create:Finished chr19:0-59128983|*
INFO:cooler.create:Writing indexes
INFO:cooler.create:Writing info
INFO:cooler.cli.zoomify:Recursively aggregating "/mnt/bioadhoc-temp/Groups/vd-ay/jreyna/projects/dchallenge/results/main/loop_catalog/processing/hg38_to_hg19/liftover/lgs301315.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.5kb.cool"
INFO:cooler.cli.zoomify:Writin

  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

  scale = nzmarg.mean()

INFO:cooler.cli.zoomify:Balancing zoom level with bin size 25000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/lgs301315.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.mcool::resolutions/25000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

  scale = nzmarg.mean()

INFO:cooler.cli.zoomify:Balancing zoom level with bin size 50000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/lgs301315.GSE116193.Homo_Sapiens.H3K27ac.b1.hg19.mcool::resolutions/50000"
  return _methods._mean(a, axis=axis, dtype=dtype,

  ret = ret.dtype.type(ret / rcount)

  scale = nzmarg.mean()

INFO:cooler.cli.zoomify:Balancing zoom level with bin size 100000
INFO:cooler.cli.balance:Balancing "results/main/loop_catalog/processing/hg38_to_hg19/liftover/fit_hp/lgs301